In [1]:
import os
import torch
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoConfig
from sentence_transformers import SentenceTransformer

load_dotenv()

def login_to_huggingface() -> None:
    login(os.environ.get("HUGGINGFACE_API_KEY"))
    return

login_to_huggingface()

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/qcqced/.cache/huggingface/token
Login successful


In [3]:
test_model = torch.load('./saved/arxiv_clm_4096_llama2_7b_hf_state_dict.pth')
for key in test_model.keys():
    print(key)

In [6]:
list(test_model.keys())[-1]

In [ ]:
model = AutoModelForCausalLM.from_config(AutoConfig.from_pretrained('meta-llama/Llama-2-7b-hf'))

for k in model.state_dict().keys():
    print(k)

In [ ]:
""" rename model attr name  """

In [13]:
import re

def convert_to_latex(text: str) -> str:
    pattern = r'(\b\w+\b)(\d*)(?=\()' 
    return re.sub(pattern, r'\\text{\1}_{\2}', text)

example_text = """FFN(x) = max(0, xW1 + b1)W2 + b2 (2)"""

latex_lines = convert_to_latex(example_text.strip())
print(latex_lines)

In [2]:
import os
import pandas as pd

In [5]:
base_path = "dataset_class/datafolder/arxiv_qa/partition/"
df_list = os.listdir(base_path)

df = pd.DataFrame(columns=['paper_id', 'doc_id', 'title', 'doc'])
for sub_url in df_list:
    sub_df = pd.read_csv(base_path + sub_url)
    df = pd.concat([df, sub_df])

df

,paper_id,doc_id,title,doc,question
0,2307.12976,2307.12976_0,Evaluating the Ripple Effects of Knowledge Edi...,Evaluating the Ripple Effects of Knowledge Edi...,NaN
1,2307.12976,2307.12976_1,Evaluating the Ripple Effects of Knowledge Edi...,Abstract,NaN
2,2307.12976,2307.12976_2,Evaluating the Ripple Effects of Knowledge Edi...,Modern language models capture a large body of...,NaN
3,2307.12976,2307.12976_3,Evaluating the Ripple Effects of Knowledge Edi...,"construct RIPPLEEDITS, a diagnos- tic benchmar...",NaN
4,2307.12976,2307.12976_4,Evaluating the Ripple Effects of Knowledge Edi...,Figure 1: Illustration of the evaluation scope...,NaN
...,...,...,...,...,...
58,2303.11749,2303.11749_58,Detecting Everything in the Open World: Toward...,<table><caption>Table 5. Comparison with exist...,NaN
59,2303.11749,2303.11749_59,Detecting Everything in the Open World: Toward...,<table><caption>Table 6. Ablation study on reg...,NaN
60,2303.11749,2303.11749_60,Detecting Everything in the Open World: Toward...,<table><thead><th>decouple</th><th>proposal ge...,NaN
61,2303.11749,2303.11749_61,Detecting Everything in the Open World: Toward...,"pij = 1 1 + \text{exp}_{}(−zT ijej/τ ) /πγ j ,...",NaN


In [6]:
output_path = 'dataset_class/datafolder/arxiv_qa/total/total_paper_chunk.csv'
df.to_csv(output_path, index=False)

In [4]:
""" load dataset for testing make prompt to generate question-document pairs dataset """
import os
import numpy as np
import pandas as pd
import google.generativeai as genai

from tqdm.auto import tqdm
from generate_question.generate_question import google_gemini_api


df = pd.read_csv('dataset_class/datafolder/arxiv_qa/total/total_paper_chunk.csv')
df

/var/folders/l1/l_9fgbw95hl79rj750047d1c0000gn/T/ipykernel_1740/639598189.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset_class/datafolder/arxiv_qa/total/total_paper_chunk.csv')


,paper_id,doc_id,title,doc,question
0,2307.12976,2307.12976_0,Evaluating the Ripple Effects of Knowledge Edi...,Evaluating the Ripple Effects of Knowledge Edi...,NaN
1,2307.12976,2307.12976_1,Evaluating the Ripple Effects of Knowledge Edi...,Abstract,NaN
2,2307.12976,2307.12976_2,Evaluating the Ripple Effects of Knowledge Edi...,Modern language models capture a large body of...,NaN
3,2307.12976,2307.12976_3,Evaluating the Ripple Effects of Knowledge Edi...,"construct RIPPLEEDITS, a diagnos- tic benchmar...",NaN
4,2307.12976,2307.12976_4,Evaluating the Ripple Effects of Knowledge Edi...,Figure 1: Illustration of the evaluation scope...,NaN
...,...,...,...,...,...
242630,2303.11749,2303.11749_58,Detecting Everything in the Open World: Toward...,<table><caption>Table 5. Comparison with exist...,NaN
242631,2303.11749,2303.11749_59,Detecting Everything in the Open World: Toward...,<table><caption>Table 6. Ablation study on reg...,NaN
242632,2303.11749,2303.11749_60,Detecting Everything in the Open World: Toward...,<table><thead><th>decouple</th><th>proposal ge...,NaN
242633,2303.11749,2303.11749_61,Detecting Everything in the Open World: Toward...,"pij = 1 1 + \text{exp}_{}(−zT ijej/τ ) /πγ j ,...",NaN


In [5]:
df = df[1500:2000]
question = [google_gemini_api(title=row['title'], context=row['doc'], foundation_model='gemini-1.5-flash') for i, row in tqdm(df.iterrows(), total=len(df))]
df['question'] = question
df

  0%|          | 0/500 [00:00<?, ?it/s]

500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
500 An internal 

KeyboardInterrupt: 

In [ ]:
output_path = 'dataset_class/datafolder/arxiv_qa/total/1500_2000_test_q_doc.csv'
df.to_csv(output_path, index=False)

In [ ]:
def google_gemini_api(
    prompt: str,
    foundation_model: str = 'gemini-pro',
    temperature: float = 0
) -> str:
    """ make Arxiv Questioning & Answering dataset function with Google AI Gemini API

    As you run this function before, you must set up the your own Google API key for the Gemini API.
    you can use the gemini-pro-api for free with the Google API key.

    we will use the Zero-Shot Learning for generating the QA dataset from the given paper link.
    Args:
        prompt: str, input prompt for generating the question
        foundation_model (str): The foundation model for extracting food ingredients from the given text,
                                default is 'gemini-pro'
        temperature (float): default 0.0, the temperature value for the diversity of the output text
                             (if you set T < 1.0, the output text will be more deterministic, sharpening softmax dist)
                             (if you set T > 1.0, the output text will be more diverse, flattening softmax dist)

    References:
        https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/quickstart_colab.ipynb?hl=ko#scrollTo=HTiaTu6O1LRC
        https://ai.google.dev/gemini-api/docs/models/gemini?hl=ko
        https://ai.google.dev/gemini-api/docs/get-started/python?hl=ko&_gl=1*7ufqxk*_up*MQ..*_ga*MTk2ODk3NDQyNi4xNzE0OTIwMjcw*_ga_P1DBVKWT6V*MTcxNDkyMDI2OS4xLjAuMTcxNDkyMDI2OS4wLjAuOTQwNDMwMTE.
        https://ai.google.dev/gemini-api/docs/quickstart?hl=ko&_gl=1*12k4ofq*_up*MQ..*_ga*MTk2ODk3NDQyNi4xNzE0OTIwMjcw*_ga_P1DBVKWT6V*MTcxNDkyMDI2OS4xLjAuMTcxNDkyMDI2OS4wLjAuOTQwNDMwMTE.
        https://ai.google.dev/api/python/google/generativeai/GenerativeModel?_gl=1*1ajz3qu*_up*MQ..*_ga*MTk2ODk3NDQyNi4xNzE0OTIwMjcw*_ga_P1DBVKWT6V*MTcxNDkyNDAyOC4yLjAuMTcxNDkyNDAyOC4wLjAuMTkwOTQyMjU0#generate_content
    """
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)

    model = genai.GenerativeModel(foundation_model)
    generation_config = genai.types.GenerationConfig(
        candidate_count=1,
        temperature=temperature
    )

    datasets = ''
    try:
        get_random_sleep(8, 10)  # for avoiding the "call api limit"
        response = model.generate_content(
            contents=prompt,
            generation_config=generation_config,
        )
        datasets = response.text

    except Exception as e:
        print(e)

    return datasets


In [30]:
""" code cell for prompt testing """

query = """You're a question machine.\nGroup the context and title together with the same number, read each group of the context and title given above and generate the appropriate question for each group.\nPlease only return the questions' text, and the number of questions should be between 1 and 5 per individual group.\nThe total number of tokens per individual group should be no more than 100 tokens.\nThe sum of all groups tokens for all questions should be same as your output's max token length.\nIf you want to ask multiple questions, please separate them with spaces without newlines.\nSeparate questions for different groups with line breaks.\nQuestions should capture the features or characteristics of the given context.\nThe purpose of asking you to create questions is to create a dataset of question-document pairs.\nPlease create with purpose and generate creative, informative, and diverse questions.\nDo not return questions that are too similar to each other or too general."""

titles, contexts = df.title.tolist()[0:10], df.doc.tolist()[0:10]
dataset = list(zip(titles, contexts))

prompt = ""
chunk_size = 10
for i in range(0, len(dataset), chunk_size):
    for j, data in enumerate(dataset[i:i+chunk_size]):
        title, context = data
        prompt += f"[title{j}]\n{title}\n\n[context{j}]\n{context}\n\n"
    
    prompt += query
    print(prompt)
    result = google_gemini_api(
        prompt=prompt,
        foundation_model="gemini-1.5-flash"
    )
    
result

[title0]
Evaluating the Ripple Effects of Knowledge Editing in Language Models.pdf

[context0]
Evaluating the Ripple Effects of Knowledge Editing in Language Models
Roi Cohen1 Eden Biran1 Ori Yoran1 Amir Globerson1,2 Mor Geva1,2, 1Blavatnik School of Computer Science, Tel Aviv University 2Google Research {roi1, edenbiran, oriy}@mail.tau.ac.il, {gamir, morgeva}@tauex.tau.ac.il

[title1]
Evaluating the Ripple Effects of Knowledge Editing in Language Models.pdf

[context1]
Abstract

[title2]
Evaluating the Ripple Effects of Knowledge Editing in Language Models.pdf

[context2]
Modern language models capture a large body of factual knowledge. However, some facts can be incorrectly induced or become obsolete over time, resulting in factually in- correct generations. This has led to the de- velopment of various editing methods that allow updating facts encoded by the model. Evaluation of these methods has primarily focused on testing whether an individual fact has been successfully injected, 

'What is the main focus of this research paper?\n\nWhat are the limitations of existing evaluation methods for knowledge editing in language models?\n\nWhat is the proposed solution to address the limitations of existing evaluation methods?\n\nWhat are the key features of the RIPPLEEDITS benchmark?\n\nWhat are the six evaluation criteria proposed in the paper to assess the ripple effects of knowledge editing? \n\nWhat are the main findings of the evaluation of prominent editing methods on RIPPLEEDITS?\n\nWhat is the main contribution of this research paper? \n'

In [50]:
""" code cell 2 for second version of prompt testing

first ver: title, context
second ver: context
"""

query = """You're a question machine.\nThe given text has a number of contexts numbered. Create questions that are appropriate for each context.\nQuestions should capture the features or characteristics of the given context.\nThe purpose of asking you to create questions is to create a dataset of question-document pairs.\nPlease create with purpose and generate creative, informative, and meaningful questions.\nDo not return questions that are too similar to each other or too general.\nPlease only return the questions' text, and the number of questions should be between 1 and 5 per single individual context group. If you want to ask more than one questions about single context, please separate them with space, not newlines.\nEach context's questions should be no more than 100 tokens.\nSeparate questions for different context group with line breaks."""


dataset = df.doc.tolist()[0:10]

prompt = ""
chunk_size = 10
for i in range(0, len(dataset), chunk_size):
    for j, data in enumerate(dataset[i:i+chunk_size]):
        prompt += f"[context{j}]\n{data}\n\n"
    
    prompt += query
    print(prompt)
    result = google_gemini_api(
        prompt=prompt,
        foundation_model="gemini-1.5-flash"
    )
    
result

[context0]
Evaluating the Ripple Effects of Knowledge Editing in Language Models
Roi Cohen1 Eden Biran1 Ori Yoran1 Amir Globerson1,2 Mor Geva1,2, 1Blavatnik School of Computer Science, Tel Aviv University 2Google Research {roi1, edenbiran, oriy}@mail.tau.ac.il, {gamir, morgeva}@tauex.tau.ac.il

[context1]
Abstract

[context2]
Modern language models capture a large body of factual knowledge. However, some facts can be incorrectly induced or become obsolete over time, resulting in factually in- correct generations. This has led to the de- velopment of various editing methods that allow updating facts encoded by the model. Evaluation of these methods has primarily focused on testing whether an individual fact has been successfully injected, and if sim- ilar predictions for other subjects have not changed. Here we argue that such evaluation is limited, since injecting one fact (e.g. “Jack Depp is the son of Johnny Depp”) introduces a “ripple effect” in the form of additional facts that the

''